In [0]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# import smote
# import rus
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_app = pd.read_csv('/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/기계학습 프로젝트/#data/application_train.csv', na_values='n/a', index_col="SK_ID_CURR")

In [0]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Columns: 121 entries, TARGET to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 286.2+ MB


Data Exploration and EDA are in complex version

# 1.1. Fill the missing values

In [0]:
def missing_values_table(df):
    
    # 전체 데이터프레임의 누락된 값 계산
    # Total missing values
    mis_val = df.isnull().sum()
    
    # 누락된 값의 백분율 계산
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # 표 작성
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    # column명 재정의
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    # 내림차순으로 표정렬
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    
    # 요약 정보 출력
    # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
    
    # 데이터프레임 return
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [0]:
missing_df = missing_values_table(df_app); missing_df

Your selected dataframe has 121 columns.
There are 67 columns that have missing values.


Missing Values  % of Total Values
COMMONAREA_MEDI                   214865               69.9
COMMONAREA_AVG                    214865               69.9
COMMONAREA_MODE                   214865               69.9
NONLIVINGAPARTMENTS_MEDI          213514               69.4
NONLIVINGAPARTMENTS_MODE          213514               69.4
...                                  ...                ...
EXT_SOURCE_2                         660                0.2
AMT_GOODS_PRICE                      278                0.1
AMT_ANNUITY                           12                0.0
CNT_FAM_MEMBERS                        2                0.0
DAYS_LAST_PHONE_CHANGE                 1                0.0

[67 rows x 2 columns]

In [0]:
''' 
 Exploring all the data, OWN_CAR_AGE & FLAG_OWN_CAR has highly related.
 Also using common knowledge, if you dont have a car, the age of your owned car should be zero.
 EXT_SOURCE_! is sources from the unkown credit bureau'''
#차 소유하지 않았으면 OWN_CAR_AGE = 0 넣기 & nan = 0 넣기
df_app['OWN_CAR_AGE']= df_app['OWN_CAR_AGE'].where(df_app['FLAG_OWN_CAR'] == 'Y', 0)
df_app['OWN_CAR_AGE'] = df_app['OWN_CAR_AGE'].where(df_app['OWN_CAR_AGE'] != np.nan, 0)
df_app['OWN_CAR_AGE'].isnull().sum()

5

In [0]:
##drop all the features whose proportion of missing values is over 50

missing_over_50 = missing_df[missing_df['% of Total Values']>50]; missing_over_50
df_app = df_app.drop(df_app[missing_over_50.index.tolist()], axis=1)
df_app.info() # now we have 80 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 80 columns):
TARGET                          307511 non-null int64
NAME_CONTRACT_TYPE              307511 non-null object
CODE_GENDER                     307511 non-null object
FLAG_OWN_CAR                    307511 non-null object
FLAG_OWN_REALTY                 307511 non-null object
CNT_CHILDREN                    307511 non-null int64
AMT_INCOME_TOTAL                307511 non-null float64
AMT_CREDIT                      307511 non-null float64
AMT_ANNUITY                     307499 non-null float64
AMT_GOODS_PRICE                 307233 non-null float64
NAME_TYPE_SUITE                 306219 non-null object
NAME_INCOME_TYPE                307511 non-null object
NAME_EDUCATION_TYPE             307511 non-null object
NAME_FAMILY_STATUS              307511 non-null object
NAME_HOUSING_TYPE               307511 non-null object
REGION_POPULATION_RELATIVE      307511 non-null float64
D

In [0]:
# dataframe of nulls

#replace XNA and blank
df_app = df_app.replace('XNA', np.nan)
df_app = df_app.replace(' ', np.nan)

#find the index that has nulls
missing_values_table(df_app)

Your selected dataframe has 80 columns.
There are 28 columns that have missing values.


Missing Values  % of Total Values
FLOORSMAX_MEDI                        153020               49.8
FLOORSMAX_AVG                         153020               49.8
FLOORSMAX_MODE                        153020               49.8
YEARS_BEGINEXPLUATATION_AVG           150007               48.8
YEARS_BEGINEXPLUATATION_MODE          150007               48.8
YEARS_BEGINEXPLUATATION_MEDI          150007               48.8
TOTALAREA_MODE                        148431               48.3
EMERGENCYSTATE_MODE                   145755               47.4
OCCUPATION_TYPE                        96391               31.3
EXT_SOURCE_3                           60965               19.8
ORGANIZATION_TYPE                      55374               18.0
AMT_REQ_CREDIT_BUREAU_QRT              41519               13.5
AMT_REQ_CREDIT_BUREAU_MON              41519               13.5
AMT_REQ_CREDIT_BUREAU_WEEK             41519               13.5
AMT_REQ_CREDIT_BUREAU_DAY              41519               13.5
AMT_REQ_CREDIT_BUREAU_HOUR             41519               13.5
AMT_REQ_CREDIT_BUREAU_YEAR             41519               13.5
NAME_TYPE_SUITE                         1292                0.4
OBS_30_CNT_SOCIAL_CIRCLE                1021                0.3
DEF_30_CNT_SOCIAL_CIRCLE                1021                0.3
OBS_60_CNT_SOCIAL_CIRCLE                1021                0.3
DEF_60_CNT_SOCIAL_CIRCLE                1021                0.3
EXT_SOURCE_2                             660                0.2
AMT_GOODS_PRICE                          278                0.1
AMT_ANNUITY                               12                0.0
CODE_GENDER                                4                0.0
CNT_FAM_MEMBERS                            2                0.0
DAYS_LAST_PHONE_CHANGE                     1                0.0

In [0]:
#eliminate rows if % of nulls <= 1%
drop_list = ['NAME_TYPE_SUITE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 
             'DEF_60_CNT_SOCIAL_CIRCLE', 'EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 
             'CODE_GENDER', 'CNT_FAM_MEMBERS', 'DAYS_LAST_PHONE_CHANGE']
for index in drop_list:
  df_app = df_app[df_app[index].notna()]
missing_values_table(df_app)

Your selected dataframe has 80 columns.
There are 17 columns that have missing values.


Missing Values  % of Total Values
FLOORSMAX_MEDI                        151676               49.8
FLOORSMAX_AVG                         151676               49.8
FLOORSMAX_MODE                        151676               49.8
YEARS_BEGINEXPLUATATION_AVG           148685               48.8
YEARS_BEGINEXPLUATATION_MODE          148685               48.8
YEARS_BEGINEXPLUATATION_MEDI          148685               48.8
TOTALAREA_MODE                        147131               48.3
EMERGENCYSTATE_MODE                   144474               47.4
OCCUPATION_TYPE                        95433               31.3
EXT_SOURCE_3                           60251               19.8
ORGANIZATION_TYPE                      54852               18.0
AMT_REQ_CREDIT_BUREAU_HOUR             41108               13.5
AMT_REQ_CREDIT_BUREAU_DAY              41108               13.5
AMT_REQ_CREDIT_BUREAU_WEEK             41108               13.5
AMT_REQ_CREDIT_BUREAU_MON              41108               13.5
AMT_REQ_CREDIT_BUREAU_QRT              41108               13.5
AMT_REQ_CREDIT_BUREAU_YEAR             41108               13.5

In [0]:
# eliminate outliers(row-wise): AMT_INCOME_TOTAL, AMT_REQ_CREDIT_BUREAU_QRT

# find the outlier's index and put na value
def find_outlier_put_na(data, col, no):
  ind = data.loc[data[col]==no].index
  data[col].loc[ind] = np.nan

find_outlier_put_na(df_app, 'AMT_INCOME_TOTAL', 117000000.0)
find_outlier_put_na(df_app, 'AMT_INCOME_TOTAL', 18000090.0)
find_outlier_put_na(df_app, 'AMT_REQ_CREDIT_BUREAU_QRT', 261.0)
find_outlier_put_na(df_app, 'DAYS_EMPLOYED', 365243)

#drop na in row-wise
df_app = df_app[df_app.AMT_INCOME_TOTAL.notna()]
df_app = df_app[df_app.AMT_REQ_CREDIT_BUREAU_QRT.notna()]
df_app = df_app[df_app.DAYS_EMPLOYED.notna()]

In [0]:
#fill na with 0 if numerical, and make another category for nulls or put mode if categorical

missings = missing_values_table(df_app)
numerical_list = []
categorical_list=[]

for index in missings.index.tolist():
    print(index)
    if df_app[index].dtype=='float64':
        numerical_list.append(index)
    else:
        categorical_list.append(index)

###numerical###
df_app[numerical_list] = df_app[numerical_list].fillna(0)
'''imputer_mean = SimpleImputer(strategy="mean")
df_app[numerical_list] = imputer_mean.fit_transform(df_app[numerical_list])
'''

###categorical###
df_app.OCCUPATION_TYPE = df_app.OCCUPATION_TYPE.fillna('Independent')
df_app.ORGANIZATION_TYPE = df_app.ORGANIZATION_TYPE.fillna('Independent')
df_app.EMERGENCYSTATE_MODE = df_app.EMERGENCYSTATE_MODE.fillna('Independent')

Your selected dataframe has 80 columns.
There are 10 columns that have missing values.
FLOORSMAX_AVG
FLOORSMAX_MODE
FLOORSMAX_MEDI
YEARS_BEGINEXPLUATATION_AVG
YEARS_BEGINEXPLUATATION_MODE
YEARS_BEGINEXPLUATATION_MEDI
TOTALAREA_MODE
EMERGENCYSTATE_MODE
OCCUPATION_TYPE
EXT_SOURCE_3


In [0]:
missing_values_table(df_app) # now we have no nulls

Your selected dataframe has 80 columns.
There are 0 columns that have missing values.


Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []

# 1.2. Preprocessing

In [0]:
### log scale ###
df_app['AMT_ANNUITY'] = np.log(df_app['AMT_ANNUITY'] )
df_app['AMT_CREDIT'] = np.log(df_app['AMT_CREDIT'])
df_app['AMT_INCOME_TOTAL'] = np.log(df_app['AMT_INCOME_TOTAL'])
df_app['AMT_GOODS_PRICE'] = np.log(df_app['AMT_GOODS_PRICE'])

In [0]:
# recategorize if # of categoreis >= 4

####NAME_TYPE_SUITE_app####
#Others : Other_A, Other_B, Group of people
#Heart : Children, Family, (Spouse, partner)
#0 : na values

df_app["NAME_TYPE_SUITE"] = df_app["NAME_TYPE_SUITE"].replace('Other_A','Others').replace('Other_B','Others').replace('Group of people','Others')
df_app["NAME_TYPE_SUITE"] = df_app["NAME_TYPE_SUITE"].replace('Children','Heart').replace('Family','Heart').replace('Spouse, partner','Heart')
#df_merged["NAME_TYPE_SUITE_app"].fillna(0,inplace=True)`

#### NAME_INCOME_TYPE #### 
# Working : Commercial associate, State servant, Businessman, working
# not_working : Student, Maternity leave, Unemployed
# Pensioner

df_app["NAME_INCOME_TYPE"] = df_app["NAME_INCOME_TYPE"].replace('Commercial associate','Working').replace('State servant','Working').replace('Businessman','Working')
df_app["NAME_INCOME_TYPE"] = df_app["NAME_INCOME_TYPE"].replace('Student','not_working').replace('Maternity leave','not_working').replace('Unemployed','not_working')

#### NAME_EDUCATION_TYPE ####
#High: Academic degree, Higher education
#Middle : Secondary / secondary special, Incomplete higher 
#Lower : Lower secondary

df_app["NAME_EDUCATION_TYPE"] = df_app["NAME_EDUCATION_TYPE"].replace('Secondary / secondary special','Middle').replace('Incomplete higher','Middle')
df_app["NAME_EDUCATION_TYPE"] = df_app["NAME_EDUCATION_TYPE"].replace('Academic degree','High').replace('Higher education','High')
df_app["NAME_EDUCATION_TYPE"] = df_app["NAME_EDUCATION_TYPE"].replace('Lower secondary','Lower')


#### NAME_FAMILY_STATUS ####
# Ect: Single / not married, Separated, 'Widow
# Civil marriage
# Married
# 0 : unknown

df_app["NAME_FAMILY_STATUS"] = df_app["NAME_FAMILY_STATUS"].replace('Single / not married','Etc').replace('Separated','Etc').replace('Widow','Etc')
df_app["NAME_FAMILY_STATUS"] = df_app["NAME_FAMILY_STATUS"].replace('Unknown','0')


###OCCUPATION_TYPE###
df_app["OCCUPATION_TYPE"] = df_app["OCCUPATION_TYPE"].replace('Waiters/barmen staff','Low').replace('Low-skill Laborers','Low').replace('Drivers','Low').replace('Cleaning staff','Low').replace('Laborers','Low').replace('Cooking staff','Low').replace('Security staff','Low')
df_app["OCCUPATION_TYPE"] = df_app["OCCUPATION_TYPE"].replace('HR staff','Middle').replace('IT staff','Middle').replace('Managers','Middle').replace('Core staff','Middle').replace('Sales staff','Middle').replace('Private service staff','Middle').replace('Secretaries','Middle').replace('Realty agents','Middle')
df_app["OCCUPATION_TYPE"] = df_app["OCCUPATION_TYPE"].replace('Medicine staff','High').replace('Accountants','High').replace('High skill tech staff','High')


######ORGANIZATION_TYPE######
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Industry: type 8','Industry').replace('Industry: type 10','Industry').replace('Industry: type 13','Industry').replace('Industry: type 6','Industry').replace('Industry: type 12','Industry').replace('Industry: type 4','Industry').replace('Industry: type 9','Industry').replace('Industry: type 3','Industry')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Industry: type 11','Industry').replace('Industry: type 7','Industry').replace('Industry: type 1','Industry').replace('Industry: type 5','Industry').replace('Industry: type 2','Industry')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Trade: type 5','Trade').replace('Trade: type 4','Trade').replace('Trade: type 1','Trade').replace('Trade: type 6','Trade').replace('Trade: type 2','Trade').replace('Trade: type 3','Trade').replace('Trade: type 7','Trade')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Business Entity Type 1','Business').replace('Business Entity Type 2','Business').replace('Business Entity Type 3','Business')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Transport: type 1','Transport').replace('Transport: type 2','Transport').replace('Transport: type 3','Transport').replace('Transport: type 4','Transport')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Kindergarten','School').replace('University','School')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Security Ministries','Security')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Government','public').replace('Military','public').replace('Police','public').replace('Postal','public')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Emergency','Medicine')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Insurance','Finance').replace('Bank','Finance').replace('Realtor','Finance')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Religion','Culture')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Telecom','Mobile')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Cleaning','Services').replace('Legal Services','Services').replace('Hotel','Services').replace('Restaurant','Services')
df_app["ORGANIZATION_TYPE"] = df_app["ORGANIZATION_TYPE"].replace('Advertising','Other').replace('Electricity','Other').replace('Agriculture','Other').replace('Housing','Other').replace('Construction','Other')


In [0]:
# All feqtures of FLAG_DOCUMET: tell us if the applicant provide the document or not
# AND the numeric feature(# of documents provided) which is sum of all these feature, also tells us enough info

df_app['FLAG_DOCUMENT'] = df_app[['FLAG_DOCUMENT_2','FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']].sum(axis=1)

#drop 
df_app =  df_app.drop(['FLAG_DOCUMENT_2','FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21'], axis=1)

One-hot

In [0]:
# order variables into dummies
df_app['REGION_RATING_CLIENT'] = df_app['REGION_RATING_CLIENT'].astype('str')
df_app['REGION_RATING_CLIENT_W_CITY'] = df_app['REGION_RATING_CLIENT_W_CITY'].astype('str')

In [0]:
def categoricalDumm(df):
    x= pd.DataFrame()
    df_1= [col for col in df.columns if df[col].dtype == "object"]
    df_2= df[df_1]
    df_dummy = pd.get_dummies(df_2)
    x= pd.concat([x,df_dummy],axis=1)
    df=df.drop(columns= df_1)
    df= pd.concat([df,x], axis=1)
    return df


In [0]:
df_app = categoricalDumm(df_app)
df_app.head()

TARGET  ...  EMERGENCYSTATE_MODE_Yes
SK_ID_CURR          ...                         
100002           1  ...                        0
100003           0  ...                        0
100004           0  ...                        0
100007           0  ...                        0
100008           0  ...                        0

[5 rows x 106 columns]

Scale the overall data

In [0]:
num_cols = df_app._get_numeric_data().columns ;num_cols.tolist()
num_ccols = num_cols.drop(['TARGET','FLAG_MOBIL']) 

In [0]:
from sklearn.preprocessing import MinMaxScaler
X_num = df_app[num_ccols]
df_app[num_ccols] = MinMaxScaler().fit_transform(X_num)


In [0]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216150 entries, 100002 to 456255
Columns: 106 entries, TARGET to EMERGENCYSTATE_MODE_Yes
dtypes: float64(104), int64(2)
memory usage: 176.5 MB


save csv(simple ver.)

In [0]:
df_app.to_csv('/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/기계학습 프로젝트/#data/df_simple_50.csv', na_rep='NaN', index='SK_ID_CURR')
#df_app.to_csv('/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/기계학습 프로젝트/#data/df_simple_40.csv', na_rep='NaN')

# 2. Dimension Reduction

PCA

In [0]:
df_simple = pd.read_csv('/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/기계학습 프로젝트/#data/df_simple_50.csv', na_values='n/a', index_col="SK_ID_CURR")

In [0]:
index = df_simple.index.tolist()

In [0]:
#Make a pca_list that consists of highly correlated features (corr>0.7)
#select the number of components such that the amount of variance that needs to be explained is greater than the percentage specified by n_components
#==> 12 features selected

pca = PCA(n_components=0.95)
pca_list = ['FLAG_DOCUMENT',
 'CODE_GENDER_F',
 'YEARS_BEGINEXPLUATATION_MODE',
 'NAME_CONTRACT_TYPE_Cash loans',
 'CODE_GENDER_M',
 'REGION_RATING_CLIENT_W_CITY_1',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_CITY_NOT_WORK_CITY',
 'EMERGENCYSTATE_MODE_Independent',
 'LIVE_REGION_NOT_WORK_REGION',
 'EMERGENCYSTATE_MODE_No',
 'FLAG_OWN_CAR_Y',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'TOTALAREA_MODE',
 'NAME_EDUCATION_TYPE_Middle',
 'CNT_CHILDREN',
 'NAME_FAMILY_STATUS_Etc',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'AMT_ANNUITY',
 'YEARS_BEGINEXPLUATATION_AVG',
 'FLOORSMAX_MEDI',
 'REGION_RATING_CLIENT_2',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'FLOORSMAX_MODE',
 'NAME_FAMILY_STATUS_Married',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'NAME_INCOME_TYPE_Working',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'FLAG_OWN_CAR_N',
 'REGION_RATING_CLIENT_3',
 'NAME_INCOME_TYPE_not_working',
 'AMT_GOODS_PRICE',
 'AMT_CREDIT',
 'FLOORSMAX_AVG',
 'REG_CITY_NOT_WORK_CITY',
 'NAME_TYPE_SUITE_Unaccompanied',
 'REGION_RATING_CLIENT_W_CITY_3',
 'REGION_RATING_CLIENT_1',
 'REGION_RATING_CLIENT_W_CITY_2',
 'FLAG_OWN_REALTY_N',
 'CNT_FAM_MEMBERS',
 'FLAG_OWN_REALTY_Y',
 'NAME_EDUCATION_TYPE_High',
 'NAME_TYPE_SUITE_Heart']
X_proj = pca.fit_transform(df_simple[pca_list])
print('principal component vec :\n', pca.components_.T)

principalDf = pd.DataFrame(data = X_proj, columns = ['pc1', 'pc2', 'pc3','pc4', 'pc5','pc6','pc7','pc8','pc9','pc10','pc11','pc12'], index= index)

principal component vec :
 [[-5.22500400e-04  4.24339121e-03  9.82646164e-04  2.30682031e-02
  -2.68402020e-03 -1.74337122e-02 -2.33377995e-03  2.14829118e-03
   2.06845979e-03  3.53574095e-03 -1.56240032e-01 -2.54341109e-03]
 [ 3.30556161e-02 -4.72362993e-01  1.90733101e-02 -7.65379813e-02
   2.10025303e-01 -1.79822847e-01 -1.44772268e-01  3.94074362e-01
  -1.25956337e-01 -3.32452460e-03  1.08499393e-03  2.79216501e-02]
 [ 4.22612448e-01  3.71727854e-02 -3.62037473e-02  1.46794941e-02
  -5.00673324e-02 -1.26235916e-02  5.00579827e-02  4.62804473e-02
  -1.96502419e-02 -6.62129620e-02  1.87458999e-04  3.62091364e-04]
 [-8.68945663e-03  7.79231553e-03 -1.68301839e-03  5.10895102e-02
  -2.91995474e-02 -5.98745327e-02 -2.41494581e-03  5.77913861e-03
  -3.81560121e-03  2.38722976e-03 -6.77062021e-01 -6.61742353e-03]
 [-3.30556161e-02  4.72362993e-01 -1.90733101e-02  7.65379813e-02
  -2.10025303e-01  1.79822847e-01  1.44772268e-01 -3.94074362e-01
   1.25956337e-01  3.32452460e-03 -1.08499393

In [0]:
df_simple.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216150 entries, 100002 to 456255
Columns: 106 entries, TARGET to EMERGENCYSTATE_MODE_Yes
dtypes: float64(104), int64(2)
memory usage: 176.5 MB


Feature selection
-using 6 methods

In [0]:
X = df_simple.drop(['TARGET'], axis=1)
y = df_simple.TARGET
feature_name = X.columns.tolist()

In [0]:
###Pearson###
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-50:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [0]:
cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')

50 selected features


In [0]:
###chi-2###
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(X)
chi_selector = SelectKBest(chi2, k=50)
chi_selector.fit(X, y)

SelectKBest(k=50, score_func=<function chi2 at 0x7f82b36c4400>)

In [0]:
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

50 selected features


In [0]:
###RFE###
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=50, step=10, verbose=5)
rfe_selector.fit(X, y)

Fitting estimator with 105 features.
Fitting estimator with 95 features.
Fitting estimator with 85 features.
Fitting estimator with 75 features.
Fitting estimator with 65 features.
Fitting estimator with 55 features.


RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=50, step=10, verbose=5)

In [0]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(list(rfe_feature))

50 selected features
[0, 1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 18, 20, 24, 25, 29, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 46, 47, 50, 51, 52, 58, 59, 61, 68, 71, 76, 77, 78, 80, 89, 90, 95, 96, 101, 102, 103, 104]


In [0]:
###Logistic Regression###
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embedded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), '1.25*median')
embedded_lr_selector.fit(X, y)

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l2',
                                             random_state=None, solver='lbfgs',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False,
                threshold='1.25*median')

In [0]:
embedded_lr_support = embedded_lr_selector.get_support()
embedded_lr_feature = X.loc[:,embedded_lr_support].columns.tolist()
print(str(len(embedded_lr_feature)), 'selected features')

50 selected features


In [0]:
###Random Forest###
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embedded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=50), threshold='1.25*median')
embedded_rf_selector.fit(X, y)

embedded_rf_support = embedded_rf_selector.get_support()
embedded_rf_feature = X.loc[:,embedded_rf_support].columns.tolist()
print(str(len(embedded_rf_feature)), 'selected features')

36 selected features


In [0]:
###Light GBM###
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embedded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
embedded_lgb_selector.fit(X, y)

embedded_lgb_support = embedded_lgb_selector.get_support()
embedded_lgb_feature = X.loc[:,embedded_lgb_support].columns.tolist()
print(str(len(embedded_lgb_feature)), 'selected features')

43 selected features


In [0]:
########################## SUMMARY ##########################
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 50
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)

# final feature selection (# of Trues for 6 feature selection models >= 4)

feature_selection_df = feature_selection_df.drop(feature_selection_df.index[34:], axis=0)
feature_selection_df

Feature  Pearson  ...  LightGBM  Total
1                    TOTALAREA_MODE     True  ...      True      6
2                    FLOORSMAX_MODE     True  ...      True      6
3                      EXT_SOURCE_3     True  ...      True      6
4                      EXT_SOURCE_2     True  ...      True      6
5            DAYS_LAST_PHONE_CHANGE     True  ...      True      6
6                   DAYS_ID_PUBLISH     True  ...      True      6
7                     DAYS_EMPLOYED     True  ...      True      6
8                        DAYS_BIRTH     True  ...      True      6
9        NAME_FAMILY_STATUS_Married     True  ...      True      5
10                   FLOORSMAX_MEDI     True  ...      True      5
11                  FLAG_WORK_PHONE    False  ...      True      5
12                   FLAG_OWN_CAR_Y     True  ...      True      5
13         DEF_30_CNT_SOCIAL_CIRCLE     True  ...      True      5
14                DAYS_REGISTRATION     True  ...      True      5
15                    CODE_GENDER_F     True  ...      True      5
16       AMT_REQ_CREDIT_BUREAU_YEAR     True  ...      True      5
17                 AMT_INCOME_TOTAL     True  ...      True      5
18                  AMT_GOODS_PRICE     True  ...      True      5
19                       AMT_CREDIT     True  ...      True      5
20     YEARS_BEGINEXPLUATATION_MODE     True  ...      True      4
21     YEARS_BEGINEXPLUATATION_MEDI     True  ...      True      4
22      YEARS_BEGINEXPLUATATION_AVG     True  ...      True      4
23           REG_CITY_NOT_LIVE_CITY     True  ...      True      4
24    REGION_RATING_CLIENT_W_CITY_3     True  ...     False      4
25    REGION_RATING_CLIENT_W_CITY_1     True  ...     False      4
26           REGION_RATING_CLIENT_3     True  ...     False      4
27       REGION_POPULATION_RELATIVE     True  ...      True      4
28  ORGANIZATION_TYPE_Self-employed     True  ...      True      4
29         ORGANIZATION_TYPE_School     True  ...      True      4
30             OCCUPATION_TYPE_High     True  ...     False      4
31         OBS_60_CNT_SOCIAL_CIRCLE    False  ...      True      4
32         OBS_30_CNT_SOCIAL_CIRCLE    False  ...      True      4
33       NAME_EDUCATION_TYPE_Middle     True  ...      True      4
34         NAME_EDUCATION_TYPE_High     True  ...     False      4

[34 rows x 8 columns]

In [0]:
final_feature = feature_selection_df.Feature.values
df_sim_final = pd.concat([df_simple['TARGET'],df_simple[final_feature]], axis=1)
df_sim_final.head()

TARGET  ...  NAME_EDUCATION_TYPE_High
SK_ID_CURR          ...                          
100002           1  ...                       0.0
100003           0  ...                       1.0
100004           0  ...                       0.0
100007           0  ...                       0.0
100008           0  ...                       0.0

[5 rows x 35 columns]

In [0]:
df_sim_final.to_csv('/content/drive/My Drive/[핀테크 과정] 머신러닝 팀플/기계학습 프로젝트/#data/df_sim_final.csv', na_rep='NaN')